In [ ]:
# SMA-EMA Backtester
# Author: Your Name
# Purpose: Backtest a simple SMA vs     
EMA crossover trading strategy

# Step 1: Import libraries
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Step 2: Set parameters
ticker = "AAPL"             # Stock symbol
start_date = "2022-01-01"   # Start date
end_date = "2025-08-01"     # End date
short_window = 20           # SMA period
long_window = 50            # EMA period
initial_cash = 10000        # Starting portfolio value

# Step 3: Download historical data
data = yf.download(ticker, start=start_date, end=end_date)
data = data[['Close']]  # Keep only closing prices

# Step 4: Compute indicators
data['SMA'] = data['Close'].rolling(window=short_window).mean()
data['EMA'] = data['Close'].ewm(span=long_window, adjust=False).mean()

# Step 5: Generate signals
data['Signal'] = 0
data['Signal'][short_window:] = \
    (data['SMA'][short_window:] > data['EMA'][short_window:]).astype(int)
data['Position'] = data['Signal'].shift(1)  # Avoid look-ahead bias

# Step 6: Compute daily returns
data['Daily_Return'] = data['Close'].pct_change()
data['Strategy_Return'] = data['Position'] * data['Daily_Return']

# Step 7: Compute cumulative portfolio value
data['Portfolio_Value'] = initial_cash * (1 + data['Strategy_Return']).cumprod()

# Step 8: Plot results
plt.figure(figsize=(12,6))
plt.plot(data['Close'], label='Close Price')
plt.plot(data['SMA'], label=f'SMA({short_window})')
plt.plot(data['EMA'], label=f'EMA({long_window})')
plt.plot(data['Portfolio_Value'], label='Portfolio Value', linestyle='--')
plt.title(f'{ticker} SMA-EMA Crossover Backtest')
plt.xlabel('Date')
plt.ylabel('Price / Portfolio Value')
plt.legend()
plt.grid(True)
plt.savefig('example_plot.png')  # Save for README
plt.show()

# Step 9: Print total return
total_return = (data['Portfolio_Value'][-1] / initial_cash - 1) * 100
print(f"Total strategy return: {total_return:.2f}%")
